In [1]:
# Some useful packages to import
import math
import copy

In [2]:
class Node:
    def __init__(self , location_size , distance_matrix , sorted_edges , all_sorted_edges , parent_constraints , extra_constraints = None):
        self.location_size = location_size # Number of locations
        self.distance_matrix = distance_matrix # distance matrix
        self.sorted_edges = sorted_edges
        self.all_sorted_edges = all_sorted_edges
        self.extra_constraints = extra_constraints
        self.constraints = self.find_constraints(parent_constraints)
        self.lower_bound = self.compute_lower_bound()
    # To find the lower bound for weight of the minimum Hamiltanian cycle
    # A cycle in a given grap G that contains every vertex of is called a Hamiltonian Cycle of G
    #--------------------------------------------------------------------------------------------------------------
      # Choose an arbitraty node. Detele the node and all its arcs
      # Fins the length of the minimum connectors for the remaining arcs
      # Add the weights of the two least weights arcs from the deleted node to the weights of the minimum connectors
    def compute_lower_bound(self):
        lb = 0
        for i in range(self.location_size):
            lower = 0
            t = 0
            for j in range(self.location_size):
                if self.constraints[i][j] == 1:
                    lower += self.distance_matrix[i][j]
                    t += 1
        tt = 0
        while t < 2:
            shortest = self.sorted_edges[i][tt]
            if self. constraints[i][shortest] == 2:
                lower += self.distance_matrix[i][shortest]
                t += 1
            tt += 1
            if tt >= self.location_size:
                lower = math.inf
                break
        lb += lower
        return lb
    
    # This function computes the constrats of the node based on the parent contrates and other contrats containing
    # by this node
    
    def find_constraints(self , parent_constraints):
        constraints = copy.deepcopy(parent_constraints)
        if self.extra_constraints == None:
            return constraints
        fr = self.extra_constraints[0]
        to = self.extra_constraints[1]
        constraints[fr][to] = self.extra_constraints[2]
        constraints[to][fr] = self.extra_constraints[2]
        for i in range (2):
            constraints = self.delete_edges(constraints)
            constraints = self.add_adges(constraints)
        return constraints
    
    # Deleting any edge from a tour leaves a path, the total weight of which must be no greater than that of the original tour. 
    # This means delete the edge whe two other egdes are adjecent to the vertex. Adding edge would create a subtour
    # To remove the extra vertices, at each step we can take a shortest path to the next unvisited vertex.
    # Thus the minimum spanning distanc matrix gives a lower bound on the optimal tour.  
    
    def delete_edges(self ,constraints):
        for i in range(self.location_size):
            t = 0
            for j in range(self.location_size):
                if (i != j) and (constraints[i][j] == 1):
                    t += 1
            if t >= 2:
                for j in range(self.location_size):
                    if constraints[i][j] == 2:
                        constraints[i][j] = 0
                        constraints[j][i] = 0
        for i in range(self.location_size):
            for j in range(self.location_size):
                t = 1
                prev = i
                fr = j
                cycle = False
                    
                nextOne = self.next_one(prev ,fr , constraints)
                while (nextOne[0]):
                    t += 1
                    next = nextOne[1]
                    if next == i:
                        cycle = True
                        break
                    if t > self.location_size:
                        break
                    prev = fr
                    fr = next
                    nextOne = self. next_one(prev ,fr , constraints )
                if (cycle) and (t < self.location_size) and (constraints[i][j] == 2):
                    constraints[i][j] = 0
                    constraints[j][i] = 0
        return constraints
    #  This function checks if all two edges adjacent to a vertex are excluded 
    # If that's the case, these edges are included 
    def add_adges(self ,constraints ):
        for i in range(self.location_size):
            t = 0
            for j in range(self.location_size):
                if constraints[i][j] == 0:
                    t += 1
            if t == self.location_size - 2:
                for j in range(self.location_size):
                    if constraints[i][j] == 2:
                        constraints[i][j] = 1
                        constraints[j][i] = 1
        return constraints
    ## This function determine whether there exists an included edge that starts in from and does not end in fr prev.
    ## end point of this edge
    def next_one(self ,prev ,fr , constraints ):
        for j in range(self.location_size):
            if ( constraints[fr][j] == 1) and (j != prev ):
                return [True ,j]
        return [False]
    # This function evaluate if a node represent a full tour by checking wheather from every vertex there are exacly 2 included edges and all
    # other edges are exacly 2 included edges and all other edges are excluded 
    def isTour(self):
        for i in range(self.location_size):
            num_zero = 0
            num_one = 0
            for j in range(self.location_size):
                if self.constraints[i][j] == 0:
                    num_zero += 1
                elif self.constraints[i][j] == 1:
                    num_one += 1
            if (num_zero != self.location_size - 2) or ( num_one != 2):
                return False
        return True
    
    #   Given a list of edges, finds the shortest subtour (check if a node contains a subtour)
    def contains_subtour(self ):
        for i in range(self.location_size):
            next = self.next_one(i,i,self.constraints )
            if next [0]:
                prev = i
                fr = next [1]
                t = 1
                next = self.next_one(prev,fr ,self.constraints )
            while next[0]:
                t += 1
                prev = fr
                fr = next[1]
                if (fr == i) and (t < self.location_size):
                    return True
                next = self.next_one(prev ,fr ,self. constraints )
                if t == self.location_size:
                    return False
        return False
    # Assuming the node represents a tour and returns the length of that given tour
    def tour_length(self ):
        length = 0
        fr = 0
        to = self.next_one(fr ,fr ,self.constraints )[1]
        for i in range(self.location_size):
            length += self.distance_matrix[fr][to]
            temp = fr
            fr = to
            to = self.next_one(temp ,to ,self.constraints )[1]
        return length
        
    # This finction determines the next constraint according to the branching strategy: lexicographic order (LG)
    def next_constraint(self ):
        for i in range(self.location_size):
            for j in range(self.location_size):
                if self.constraints[i][j] == 2:
                    return [i,j]
    # If the node represent a tour, this method returns a string with the order of the vertices in the tour
    def __str__(self):
        if self. isTour():
            result = '0'
            fr = 0
            to = None
            for j in range(self.location_size):
                if self. constraints[fr][j] == 1:
                    to = j
                    result += '-' + str(j)
                    break
            for i in range(self.location_size - 1):
                for j in range(self.location_size):
                    if (self.constraints[to][j] == 1) and (j != fr):
                        result += '-' + str(j)
                        fr = to
                        to = j
                        break
            return result
        else:
            print ('This node is not a tour')
                    
        


In [3]:
class TSP():
    def __init__(self , location_size , distance_matrix ,optimal_tour = math.isinf ):
        
     
        self.location_size = location_size
        self.distance_matrix= distance_matrix
        self.optimal_tour = optimal_tour
        self.best_node = None
        self.num_created_nodes = 0
        self.num_pruned_nodes = 0
        self.sorted_edges = self.sort_edges()
        self.all_sorted_edges = self.sort_all_edges()
        
    def get_solution(self):
        root = self.create_root()
        self. num_created_nodes += 1
        self. branch_and_bound(root)
        print('−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−')
        print('The shortest tour is:')
        print(self.best_node)
        print('It has a length of', self.optimal_tour , 'km')
        print('Number of nodes created :',self.num_created_nodes )
        print('Number of nodes pruned :',self.num_pruned_nodes )
        
    # This function sorts the edges of the ditance matrix per row return matrix where each row i contains the numbers
    # 0 <=k<= location_size - 1 in order of increasing cost of edges (i,k)
    def sort_edges(self):
        result = []
        for i in range(self.location_size):
            result.append([x for (y, x) in sorted (zip(self.distance_matrix[i],list(range(self.location_size))))])
        return result
    # This function sorts all edges of distance matrix returns list of pairs [i,j] in order of increasing costs
    def sort_all_edges(self):
        edges = []
        lengths = []
        for i in range(self.location_size):
            for j in range(i + 1, self.location_size):
                edges.append([i, j])
                lengths.append(self.distance_matrix[i][j])
        result = [z for (l, z) in sorted (zip(lengths , edges ))]
        return result
        
        
    def create_root(self):
        no_constraints = []
        for i in range(self.location_size):
            row_i = []
            for j in range(self.location_size):
                if (i != j):
                    row_i.append (2)
                else:
                    row_i.append (0)
            no_constraints.append(row_i)
        root = Node(self.location_size,self.distance_matrix ,self.sorted_edges,self.all_sorted_edges , no_constraints )
        return root
    # Branch and Bound method, for current node in tree, we compute a bound on best possible solution that we can get if we down this node. If the bound on best possible solution itself is worse than current best (best computed so far),
    # then we ignore the subtree rooted with the node. 
    # Note that the cost through a node includes two costs
    # ------1. Cost of reaching the node from the root (When we reach a node, we have this cost computed
    # ------2.  Cost of reaching an answer from current node to a leaf (We compute a bound on this 
    #       cost to decide whether to ignore subtree with this node or not).
    def branch_and_bound(self,node):
        if node.isTour():
            if node.tour_length() < self.optimal_tour:
                self.optimal_tour = node.tour_length()
                self.best_node = node
                print(self.best_node) 
                print('this is the optimal tour of length',self.optimal_tour , 'km')
        
        else:
            new_constraint = copy.copy(node.next_constraint())
            new_constraint.append(1)
            leftChild = Node(self.location_size ,self.distance_matrix ,self.sorted_edges ,
            self.all_sorted_edges ,node.constraints , new_constraint )
            new_constraint[2] = 0
            rightChild = Node(self.location_size ,self.distance_matrix ,self.sorted_edges ,self.all_sorted_edges ,node.constraints , new_constraint )
            self.num_created_nodes += 2
            if self.num_created_nodes % 401 == 0:
                print('Number of nodes created so far:',self. num_created_nodes)
                print('Number of nodes pruned so far:',self. num_pruned_nodes)
            if self.num_created_nodes % 51 == 0:
                print ('.')
            if (leftChild.contains_subtour()) or (leftChild.lower_bound > self.optimal_tour):
                leftChild = None 
                self.num_pruned_nodes += 1
            if (rightChild.contains_subtour()) or (rightChild.lower_bound > self.optimal_tour):
                rightChild = None
                self.num_pruned_nodes += 1
            if (leftChild != None) and (rightChild == None):
                self.branch_and_bound(leftChild)
            elif (leftChild == None) and (rightChild != None):
                self.branch_and_bound(rightChild)
            elif (leftChild != None) and (rightChild != None):
                if leftChild.lower_bound <= rightChild.lower_bound :
                    if leftChild.lower_bound < self.optimal_tour:
                        self.branch_and_bound(leftChild)
                    else:
                        leftChild = None
                        self.num_pruned_nodes += 1
                    if rightChild.lower_bound < 2*self.optimal_tour:
                        self.branch_and_bound(rightChild)
                    else:
                        rightChild = None
                        self.num_pruned_nodes += 1
                else:
                    if rightChild.lower_bound < self.optimal_tour:
                        self.branch_and_bound(rightChild)
                    else:
                        rightChild = None
                        self.num_pruned_nodes += 1
                    if leftChild.lower_bound < 2*self.optimal_tour:
                        self.branch_and_bound(leftChild)
                    else:
                        leftChild = None
                        self.num_pruned_nodes += 1
        
    # This function determined the next constraints using IL
    def next_constraint(self ):
        for edge in self.all_sorted_edges :
            i = edge[0]
            j = edge[1]
            if self.constraints[i][j] == 2:
                return edge
    # This function calculates the lower bound OT
    def compute_lower_bound2(self ):
        lb = 0
        onetree = np.zeros((self.location_size,self.location_size),np.int8)
        t = 0
        for i in range (1, self.location_size):
            for j in range(i + 1, self.location_size):
                if self. constraints [i][j] == 1:
                    onetree[i][j] = 1
                    onetree[j][i] = 1
                    t += 1
                    lb += self.costs[i][j]
        for edge in self. all_sorted_edges :
            if t >= self.size - 1:
                break
            i = edge[0]
            j = edge[1]
            if (self.constraints[i][j] == 2) and (i != 0):
                onetree[i][j] = 1
                onetree[j][i] = 1
            if self.onetree_contains_cycle(onetree):
                onetree[i][j] = 0
                onetree[j][i] = 0
            else:
                t += 1
                lb += self.costs[i][j]
        t = 0
        for j in range(self.size):
            if self.constraints[0][j] == 1:
                onetree[0][j] = 1
                onetree[j][0] = 1
                lb += self.costs[0][j]
                t += 1
        tt = 0
        
        while t < 2:
            shortest = self.sorted_edges[0][tt]
            if self.constraints[0][shortest] == 2:
                onetree[0][shortest] = 1
                onetree[shortest][0] = 1
                lb += self.costs[0][shortest]
                t += 1
            tt += 1
        return lb

In [4]:
  # Distance matrix
dist_matrix = [
    [   0, 90.4, 153, 305, 148, 249], # 115 St Andrews ----(0)
    [90.4,    0, 260, 299,  86.0,  161], # 67 Boshoff Street---(1)
    [ 153, 260,    0,  200,  293, 327], # 4 Paul Avenue-----(2)
    [305, 299,  200,    0, 389 ,  143], # 166 Kerk Street-----(3)
    [148,  86.0, 293 , 389,    0,  253], # 9 Margaret Street----(4)
    [249, 161,  327,  143,  253,    0]] # 16 Poort Road-----(5)

In [5]:
location_size =6 # location size based on our data

In [6]:
r = TSP(location_size, dist_matrix) # This runs the TSP class 

In [7]:
r.get_solution() # This finds the solution by printing the best tour and total distance foer the whole route

0-1-5-3-4-2-0
('this is the optimal tour of length', 1229.4, 'km')
0-1-4-5-3-2-0
('this is the optimal tour of length', 925.4, 'km')
.
0-2-3-5-1-4-0
('this is the optimal tour of length', 891.0, 'km')
−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−
The shortest tour is:
0-2-3-5-1-4-0
('It has a length of', 891.0, 'km')
('Number of nodes created :', 125)
('Number of nodes pruned :', 3)


#### (0) 115 St Andrews  -> (2)  4 Paul Avenue -> (3) 166 Kerk Street -> (5) 16 Poort Road  -> (1) 67 Boshoff Street67 ->(4) 9 Margaret (1) Street -> 115 St Andrews

In [8]:
from IPython.display import Image
img = 'Optimal_Tour.png'
Image(url=img, width=600, height=600)
